## Membros

- Amanda Oliveira
- Daniel Henrique
- Samuel Pedro

## Resumo

### Parâmetros

Será usado um kernel `rbf` em um kernel SVC. O `C` será otimizado pelo Optuna a fim de obter um resultado melhor. O `gamma` foi deixado no valor padrão que é `scale`, a fórmula referente a esse valor está disponível na documentação do scikit-learn.

### Representação adotada

A representação adotada foi escalar os valores númericos com um scaler do sklearn (no momento, um RobustScaler). Criar uma bag of words a partir do resumo. Criar uma bag of items com o elenco. Extrair o ano da data de estreia. E, por fim, descartar a história original e o titulo por não serem tão relevantes para os resultados. A coluna "adulo" foi deixada mesmo estando distribuida de maneira extremamente desuniforme, assumindo-se que o modelo irá incorporá-la melhor caso ela seja distribuída mais uniformemente.

Porém, o resultado mais efeitvo usa apenas a bag of words e a bag of items, descartando os demais fatores. Isso provavelmente está relacionado com a grande ausência de informações (NaNs) dos demais parâmetros.

### Método adotado

O método adotado foi um classificador SVC com um kernel `rbf`, já que sua acurácia se mostrou maior que o `linear`. O kernel `poly` não foi testado por falta de tempo, mas espera-se que ele seja menos preciso em grais baixos ou consideravelmente mais lento em grais altos. 

Os resultados do método Random Forest são levemente inferiores, mas conseguem incorporar mais informações para a predição.

## Pós-análise

Após algumas reflexões, percebeu-se que o modelo era altamente dependente ao resumo, o que não é algo necessariamente ruim. Por conta disso, decidimos adotar uma representação que consiste basicamente do resumo com uma limpeza aplicada por cima. Para os filmes que não se têm um resumo, usaremos um segundo modelo baseado em RandomForest para prever a categoria baseando-se nos demais parâmetros, caso esse modelo mostre-se insuficientemente preciso, substituiremos-o por uma escolha fixa do gênero comédia, já que esse é o valor estatisticamente mais comum para esse caso de borda em específico.

In [1]:
import pandas as pd
import numpy as np
from competicao_am.gerar_resultado_teste import gerar_saida_teste

In [2]:
df_treino = pd.read_csv('datasets/movies_amostra.csv')
df_predict = pd.read_csv('datasets/movies_amostra_teste_ex.csv')

df_treino.head(2)

,id,titulo,adulto,orcamento,idioma_original,popularidade,data_de_estreia,resumo,receita,duracao,genero,ator_1,ator_2,ator_3,ator_4,ator_5,dirigido_por,escrito_por_1,escrito_por_2,historia_original
0,86295,Treasure of the Yankee Zephyr,False,0,en,2.0,1981-11-28,In a lake high in the mountains of New Zealand...,0.0,108.0,Action,Ken Wahl,Lesley Ann Warren,Donald Pleasence,George Peppard,Bruno Lawrence,David Hemmings,Everett De Roche,NaN,NaN
1,289198,Redeemer,False,0,es,2.0,2014-09-18,A former hit-man for a drug cartel becomes a v...,0.0,88.0,Action,Marko Zaror,Loreto Aravena,Mauricio Diocares,José Luís Mósca,Noah Segan,Ernesto Díaz Espinoza,NaN,NaN,NaN


In [3]:
from competicao_am.preprocessamento_atributos_competicao import DataframePreprocessing

preprocessor = DataframePreprocessing(df_treino, df_treino, 'genero')

len(preprocessor.df_resumo()[0].columns)

preprocessor.df_resumo()[0].columns

Index(['resumo_aanmodderfakker', 'resumo_aaron', 'resumo_aarons',
       'resumo_aashirvad', 'resumo_aback', 'resumo_abalaba', 'resumo_abandon',
       'resumo_abandoned', 'resumo_abandoning', 'resumo_abandonment',
       ...
       'resumo_zooey', 'resumo_zookeeper', 'resumo_zooms', 'resumo_zords',
       'resumo_zulu', 'resumo_zulus', 'resumo_zvezda', 'resumo_zwariowana',
       'resumo_zxz', 'resumo_zyga'],
      dtype='object', length=18697)

## Otimização dos parâmetros

In [4]:
import optuna
import pickle # Serialização dos resultados

In [5]:
import matplotlib.pyplot as plt
from sklearn import svm

from base_am.resultado import Fold
from base_am.avaliacao import Experimento
import competicao_am.metodo_competicao as mc
import competicao_am.avaliacao_competicao as ac

## Um pouco sobre SVC's

### Kernel

O `kernel` indica o tipo de "hiper-plano" usado para separar as classes.

Os kernels a serem considerados são: `linear`, `rbf` e `poly`

#### `poly`: degree

Grau do polinômio a ser usado. Quanto maior o valor, maior o tempo gasto para o treino.

### gamma

Parâmetro usado para "hiper-planos" não lineares. Quanto maior, mais ele tenta en encaixar o modelo perfeitamente.

Valores muito altos caminhão em direção de um _overfitting_.

### C

C é o parâmetro de penalidade do erro. Ele controla o prejuízo entre paredes de decisões suaves e classificar os pontos corretamente.



Links úteis:

- [A guide to SVM parameter tuning](https://towardsdatascience.com/a-guide-to-svm-parameter-tuning-8bfe6b8a452c)

In [6]:
def experimento(metodo: mc.MetodoCompeticaoValidacaoSVM, num_trials: int = 2, otimizador:ac.OtimizacaoObjetivo = ac.OtimizacaoObjetivoSVCRbfComGamma, ml_method = None) -> Experimento:
    folds = Fold.gerar_k_folds(df_treino, val_k=5, col_classe="genero",
                                num_repeticoes=1, num_folds_validacao=4, num_repeticoes_validacao=1)

    ClasseObjetivo = ac.WrapperOtimizacaoObjetivo(otimizador, metodo=metodo)

    experimento = Experimento(folds, ml_method=ml_method, ClasseObjetivoOtimizacao=ClasseObjetivo, num_trials=num_trials)

    return experimento

def best_parameters(experiment: Experimento):
    results = []
    for study in experiment.studies_per_fold:
        results.append((study.best_trial.value, study.best_trial.params))
    results.sort()
    return list(reversed(sorted(results)))

In [7]:
import pickle
import os

version = 2

def do_experiment(name: str, metodo: mc.MetodoCompeticaoValidacaoSVM = None, num_trials: int = None, otimizador:ac.OtimizacaoObjetivo = None, ml_method = None, force: bool = False) -> Experimento:
    global version

    file_name = f'{name}__{version}.p'
    file_path = f'resultados/{file_name}'

    if not force and os.path.isfile(file_path):
        exp = None

        with open(file_path, 'rb') as exp_p:
            exp = pickle.load(exp_p)
            # Check instance here

        return exp
    
    if ml_method is None and (metodo is None or num_trials is None or otimizador is None):
        raise ValueError('To run the experiment, the appropriate parameters should be give (ml_method or metodo, num_trials and otimizador)')

    exp = experimento(metodo=metodo, num_trials=num_trials, otimizador=otimizador, ml_method=ml_method)
    macro_f1 = exp.macro_f1_avg

    with open(file_path, 'wb') as exp_p:
        # exp = pickle.dump(exp, exp_p)

    return exp

In [17]:
exp_test_rf = experimento(mc.MetodoCompeticaoValidacaoRF, 1000, ac.OtimizacaoObjetivoRandomForest)
print(exp_test_rf.macro_f1_avg)
best_parameters(exp_test_rf)
pickle.dump(exp_test_rf, open(f'resultados/exp_test_rf__{version}.p', 'wb'))


value: 0.581781805708494 and parameters: {'min_samples_split': 0.11292769985146721, 'max_features': 0.39486239619628666, 'num_arvores': 2}. Best is trial 33 with value: 0.5886801037590487.
[I 2020-08-31 20:24:26,587] Trial 918 finished with value: 0.581781805708494 and parameters: {'min_samples_split': 0.11365383423855148, 'max_features': 0.3954998979293513, 'num_arvores': 2}. Best is trial 33 with value: 0.5886801037590487.
[I 2020-08-31 20:24:26,725] Trial 921 finished with value: 0.581781805708494 and parameters: {'min_samples_split': 0.11359321035712104, 'max_features': 0.24407411191464742, 'num_arvores': 2}. Best is trial 33 with value: 0.5886801037590487.
[I 2020-08-31 20:24:26,746] Trial 922 finished with value: 0.581781805708494 and parameters: {'min_samples_split': 0.11110687184163331, 'max_features': 0.38966693047386386, 'num_arvores': 2}. Best is trial 33 with value: 0.5886801037590487.
[I 2020-08-31 20:24:26,760] Trial 919 finished with value: 0.581781805708494 and paramete

In [11]:
from sklearn import svm

ml_method = svm.SVC(kernel='rbf', gamma=0.10044264686563253,
                        C=2 ** 2.8396028413359202, random_state=2)

exp_test_svm = experimento(mc.MetodoCompeticaoValidacaoSVM, 100, ac.OtimizacaoObjetivoSVCRbfComGamma)
print(exp_test_svm.macro_f1_avg)

ost': 6.223677544448442, 'gamma': 0.06472387738634526}. Best is trial 6 with value: 0.7677174557294635.
[I 2020-08-30 20:49:42,233] Trial 1 finished with value: 0.7708926677279739 and parameters: {'exp_cost': 2.5041236393896105, 'gamma': 0.18042058714406753}. Best is trial 1 with value: 0.7708926677279739.
[I 2020-08-30 20:49:45,368] Trial 3 finished with value: 0.7686433616569905 and parameters: {'exp_cost': 6.960404841331874, 'gamma': 0.1569486239956549}. Best is trial 1 with value: 0.7708926677279739.
[I 2020-08-30 20:49:47,599] Trial 4 finished with value: 0.7698215780584456 and parameters: {'exp_cost': 3.5416659669492043, 'gamma': 0.16172065165502372}. Best is trial 1 with value: 0.7708926677279739.
[I 2020-08-30 20:49:47,925] Trial 0 finished with value: 0.7642608675095176 and parameters: {'exp_cost': 0.8816684634139268, 'gamma': 0.4210104866296426}. Best is trial 1 with value: 0.7708926677279739.
[I 2020-08-30 20:49:59,086] Trial 7 finished with value: 0.7710063655516097 and par

In [13]:
pickle.dump(exp_test_svm, open(f'resultados/exp_test_svm__{version}.p', 'wb'))

In [14]:
params = best_parameters(exp_test_svm)
params

[(0.7841233502726488,
  {'exp_cost': 1.8103213437733223, 'gamma': 0.29314287596361527}),
 (0.7742250017799,
  {'exp_cost': 2.9461571639714297, 'gamma': 0.5065359268442085}),
 (0.7711081220319707,
  {'exp_cost': 1.5848032924797064, 'gamma': 0.3393177716513029}),
 (0.7672800702317709,
  {'exp_cost': 2.364547751708324, 'gamma': 0.19360170316818714}),
 (0.7626444077251564,
  {'exp_cost': 2.066480881011902, 'gamma': 0.41917634410149274})]